In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tables
from random import shuffle
from IPython.display import clear_output
from sklearn import metrics
from tqdm import tqdm
import time
import seaborn as sns
%matplotlib inline

/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import os
USE_GPU = True
os.environ['CUDA_VISIBLE_DEVICES'] = '3' if USE_GPU else ''
EPS = 1e-15

In [5]:
lpmt_hits = pd.read_hdf('../data/lpmt_hits.h5', mode='r') 
spmt_hits = pd.read_hdf('../data/spmt_hits.h5', mode='r')
lpmt_pos = pd.read_csv('data/lpmt_pos.csv') 
spmt_pos = pd.read_csv('data/spmt_pos.csv') 
true_info = pd.read_csv('data/true_info.csv') 

In [6]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

24245329920


In [7]:
len(lpmt_hits), len(spmt_hits)

(929768817, 33958478)

In [8]:
lpmt_hits.head() # spmt_hits.head() 

,event,hitTime,isDN,pmtID
0,0,249.992615,False,14175
1,0,40.010311,False,17319
2,0,162.123199,False,16882
3,0,51.875614,False,14951
4,0,79.817497,False,10947


In [9]:
lpmt_pos.head() # spmt_pos.head()

,pmt_id,pmt_x,pmt_y,pmt_z
0,0,1.049021e+03,0.0000,19171.32
1,1,9.084785e+02,524.5103,19171.32
2,2,5.245103e+02,908.4785,19171.32
3,3,6.423399e-14,1049.0206,19171.32
4,4,-5.245103e+02,908.4785,19171.32


In [10]:
true_info.head()

,E,R,evtID,x,y,z
0,4.747791,14610.378,0,8290.7790,11995.6180,911.74286
1,3.919721,14630.141,1,11397.6320,5407.4497,-7409.08200
2,6.823932,14573.132,2,14063.3380,-3812.8540,246.65280
3,3.765940,16820.080,3,-2377.9307,-16317.7020,3315.59030
4,3.217473,13026.938,4,-8617.1170,868.5116,9730.98600


# KNN

In [11]:
total_pos = lpmt_pos.append(spmt_pos) 

In [12]:
sum(total_pos['pmt_id'] == 318069)

0

In [13]:
neibours = np.zeros((len(total_pos), 100), dtype='int32')

coord_list = []
for _, x, y, z in total_pos.itertuples(index=False):
    coord_list.append((x, y, z))

coord = np.array(coord_list)

for i, (id1, x1, y1, z1) in tqdm(enumerate(total_pos.itertuples(index=False)), total=len(total_pos)):
    dists = np.sum(np.square(coord - (x1, y1, z1)), axis=1)
    neibours[i] = np.argsort(dists)[1:101]

100%|██████████| 42691/42691 [02:36<00:00, 272.70it/s]


In [14]:
lpmt_hits['event'].unique()
event_id = 0
event_hits = lpmt_hits.loc[lpmt_hits['event'] == event_id].append(
                   spmt_hits.loc[spmt_hits['event'] == event_id])

In [15]:
# Detector 318069 not found
event_hits.loc[event_hits['pmtID'] == 318069]

,event,hitTime,isDN,pmtID
0,0,37.040001,False,318069


In [16]:
pmt_id_map = {v:i for i, v in enumerate(total_pos['pmt_id'].values)}

In [17]:
top_n = 20
dtime = 100

In [18]:
d = {}
for _, time, _, mtID in event_hits.itertuples(index=False):
    if (mtID in pmt_id_map):
        i = pmt_id_map[mtID]
        l = d.get(i, [])
        l.append(time)
        d[i] = l

d = {k: np.array(v) for k, v in d.items()}

In [19]:
values = []
for _, time, _, mtID in event_hits.itertuples(index=False):
    if (mtID not in pmt_id_map):
        values.append(0)
        continue
    i = pmt_id_map[mtID]
    value = 0 
    for j in neibours[i,0:top_n]:
        if j in d:
            value += np.sum(np.abs(time - d[j]) < dtime)
    values.append(value)

In [20]:
values = np.array(values)

In [21]:
np.mean(values[~event_hits['isDN']])

14.05354609929078

# CatBoost